# Gotham Dataset 2025: Federated learning

Welcome to this tutorial on using the Smart Cities Network Security Dataset for training a deep learning model. In this notebook, we extend our model to a federated learning (FL) setup, where training is distributed across multiple smart city IoT nodes. Instead of sharing raw data, only model updates (gradients) are exchanged, preserving privacy and reducing bandwidth usage.

For federated learning, the workflow remains similar but differs in how the model is trained. Instead of a single centralized dataset, data is distributed across multiple IoT devices (clients), and training occurs locally before aggregating updates on a central server. We’ll use Flower (FL), a popular framework for federated learning with PyTorch.

---

## Environment Setup

Now let's really begin with this tutorial!

To start working, very little is required once you have activated your Python environment (e.g. via `conda`, `virtualenv`, `pyenv`, etc). If you are running this code on Colab, there is really nothing to do except to install Flower and other dependencies. Let's first, install Flower, then the ML framework of your choice and extra dependencies you might want to use.

## Installing Flower

You can install flower very conveniently from `pip`:

In [ ]:
# if you want to install libraries, use
# !pip install package_name

We will be using the _simulation_ engine in Flower, which allows you to run a large number of clients without the overheads of manually managing devices. This is achieved via the `Simulation Engine`, the core component in Flower to run simulations efficiently.

Now, import the required libraries:

In [3]:
import numpy as np
import pandas as pd
import glob
import os
import re

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data.dataset import Dataset

import matplotlib.pyplot as plt

from tqdm import tqdm

### Import data from Google drive

The dataset can be imported using google drive. Import drive and use mount keyword to make drive as active directory. Variable basedir stores the location of folder where dataset is stored in the drive.

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# change this line your folder where the data is found
DATA_DIR = '/content/drive/MyDrive/GothamDataset2025/processed'

## Load the Partitions

To start designing a Federated Learning pipeline we need to meet one of the key properties in FL: each client has its own data partition.

### A dataset

Let's begin by constructing the dataset.

In [9]:
class GothamDataset(Dataset):

    def __init__(self, features_file, target_file, transform=None, target_transform=None):
        """
        Args:
            features_file (string): Path to the csv file with features.
            target_file (string): Path to the csv file with labels.
            transform (callable, optional): Optional transform to be applied on features.
            target_transform (callable, optional): Optional transform to be applied on labels.
        """
        self.features = pd.read_pickle(features_file, compression="gzip")
        self.labels = pd.read_pickle(target_file, compression="gzip")
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        feature = self.features.iloc[idx, :]
        label = self.labels.iloc[idx, 0]
        if self.transform:
            feature = self.transform(feature.values, dtype=torch.float32)
        if self.target_transform:
            label = self.target_transform(label, dtype=torch.int64)
        return feature, label

In [10]:
def load_partition(data_path: str, partition_id: int, batch_size: int):
    """Load training, validation and test set."""
    
    iot_devices = []
    for path in glob.glob(f"{DATA_DIR}/*_train_features.pkl"):
        match = re.search(r"([^/]+)_train_features\.pkl$", path)
        if match:
            iot_devices.append(match.group(1))
    
    iot_device = iot_devices[partition_id]
    # Get the datasets
    train_data = GothamDataset(
        features_file=f"{data_path}/{iot_device}_train_features.pkl",
        target_file=f"{data_path}/{iot_device}_train_labels.pkl",
        transform=torch.tensor,
        target_transform=torch.tensor
    )

    test_data = GothamDataset(
        features_file=f"{data_path}/{iot_device}_test_features.pkl",
        target_file=f"{data_path}/{iot_device}_test_labels.pkl",
        transform=torch.tensor,
        target_transform=torch.tensor
    )

    # Create the dataloaders - for training, validation and testing
    train_loader = torch.utils.data.DataLoader(
        dataset=train_data,
        batch_size=batch_size,
        shuffle=True
    )
    test_loader = torch.utils.data.DataLoader(
        dataset=test_data,
        batch_size=batch_size,
        shuffle=False
    )

    return train_loader, test_loader

### A DNN architecture



In [11]:
import torch.nn as nn
import torch.nn.functional as F


class DNN(nn.Module):
    def __init__(self, num_features, hidden1_size, hidden2_size, hidden3_size, num_classes):
        super(DNN, self).__init__()
        self.fc1 = nn.Linear(num_features, hidden1_size)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden1_size, hidden2_size)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden2_size, hidden3_size)
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(hidden3_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        out = self.relu3(out)
        out = self.fc4(out)
        return out

Similarly to what we did with the dataset you could inspect the model in various ways. We can, for instance, count the number of model parameters.

     

In [12]:
# Defining some input variables
n_features = 70
n_classes = 6
num_epochs = 3

# Creating a DBN
model = DNN(num_features=n_features,
            hidden1_size=128,
            hidden2_size=128,
            hidden3_size=64,
            num_classes=n_classes,
            )
num_parameters = sum(value.numel() for value in model.state_dict().values())
print(f"{num_parameters = }")

num_parameters = 34246


## Federated Learning with Flower


Next, we simulate a scenario where:
- Multiple datasets are distributed across multiple organizations.
- The model is trained collaboratively across these organizations using federated learning.

In federated learning, the process works as follows:

- The server sends the global model parameters to the client.
- The client:
    - Updates the local model with the parameters received from the server.
    - Trains the model on its local data, which changes the model parameters locally.
    - Sends the updated/changed model parameters back to the server (or, alternatively, just the gradients).


### Define a Federated Learning Client

Each IoT device (client) will perform local training and send model updates to the server. 

Each client:
- Loads its local dataset
- Trains the model locally
- Sends model updates to the central server

A Flower Client is a simple Python class with two distinct methods: `fit()` and `evaluate()`. This class will be then wrapped into a ClientApp that can be used to launch the simulation.

In [ ]:
from collections import OrderedDict
from typing import Dict, Tuple

import torch
from flwr.common import NDArrays, Scalar
from flwr.client import NumPyClient

In [ ]:
class FlowerClient(NumPyClient):
    def __init__(self, cid, model, criterion, train_loader, valid_loader, test_loader, args):
        self.cid = cid
        self.model = model
        self.criterion = criterion
        self.train_loader = train_loader
        self.valid_loader = valid_loader
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    def fit(self, parameters, config):
        """This method trains the model using the parameters sent by the
        server on the dataset of this client. At then end, the parameters
        of the locally trained model are communicated back to the server"""

        # copy parameters sent by the server into client's local model
        set_params(self.model, parameters)

        # Define the optimizer
        optim = torch.optim.SGD(self.model.parameters(), lr=0.01, momentum=0.9)

        # do local training (call same function as centralised setting)
        train(self.model, self.train_loader, optim, self.device)

        # return the model parameters to the server as well as extra info (number of training examples in this case)
        return get_params(self.model), len(self.train_loader), {}

    def evaluate(self, parameters: NDArrays, config: Dict[str, Scalar]):
        """Evaluate the model sent by the server on this client's
        local validation set. Then return performance metrics."""

        set_params(self.model, parameters)
        # do local evaluation (call same function as centralised setting)
        loss, accuracy = test(self.model, self.valid_loader, self.device)
        # send statistics back to the server
        return float(loss), len(self.valid_loader), {"accuracy": accuracy}


# Two auxhiliary functions to set and extract parameters of a model
def set_params(model, parameters):
    """Replace model parameters with those passed as `parameters`."""

    params_dict = zip(model.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.from_numpy(v) for k, v in params_dict})
    # now replace the parameters
    model.load_state_dict(state_dict, strict=True)


def get_params(model):
    """Extract model parameters as a list of NumPy arrays."""
    return [val.cpu().numpy() for _, val in model.state_dict().items()]

### The `client_fn` callback

Now, we need to construct a `ClientApp`. This can be conveniently be done by means of a `client_fn` callback that will return a `FlowerClient` that uses a specific data partition (`partition-id`).

In [ ]:
def client_fn(context: Context) -> Client:
    """Create a Flower client representing a single organization."""

    # Load model
    net = Net().to(DEVICE)

    # Load data
    # Note: each client gets a different trainloader/valloader, so each client
    # will train and evaluate on their own unique data partition
    # Read the node_config to fetch data partition associated to this node
    partition_id = context.node_config["partition-id"]
    trainloader, valloader, _ = load_datasets(partition_id=partition_id)

    # Create a single Flower client representing a single organization
    # FlowerClient is a subclass of NumPyClient, so we need to call .to_client()
    # to convert it to a subclass of `flwr.client.Client`
    return FlowerClient(net, trainloader, valloader).to_client()


# Create the ClientApp
client = ClientApp(client_fn=client_fn)

### Define the Flower ServerApp

The server coordinates training across IoT clients by aggregating model updates. A strategy sits at the core of the Federated Learning experiment. For this tutorial, let's use what is arguable the most popular strategy out there: FedAvg.


FedAvg is a good strategy to start your experimentation. The way it works is simple. FedAvg, as its name implies, derives a new version of the global model by taking the average of all the models sent by clients participating in the round.

<!-- We'll use this callback when defining the strategy in the next section. -->

##### The server_fn callback

The easiest way to create a ServerApp is by means of a server_fn callback. It has a similar signature to client_fn but, instead of returning a client object, it returns all the components needed to run the server-side logic in Flower. In this tutorial we'll keep things simple and stick to FedAvg with initialised global parameters.

In [ ]:
from flwr.common import ndarrays_to_parameters
from flwr.server import ServerApp, ServerConfig, ServerAppComponents
from flwr.server.strategy import FedAvg

In [ ]:
def server_fn(context: Context):

    # Define the strategy
    strategy = FedAvg(
        fraction_fit=0.1,  # 10% clients sampled each round to do fit()
        fraction_evaluate=0.5,  # 50% clients sample each round to do evaluate()
        min_fit_clients=10,  # Never sample less than 10 clients for training
        min_evaluate_clients=5,  # Never sample less than 5 clients for evaluation
        min_available_clients=10,  # Wait until all 10 clients are available
    )

    num_rounds = 5
    
    # Construct ServerConfig
    config = ServerConfig(num_rounds=num_rounds)

    # Wrap everything into a `ServerAppComponents` object
    return ServerAppComponents(strategy=strategy, config=config)


# Create your ServerApp
server_app = ServerApp(server_fn=server_fn)

### Resource Configuration

To control resource usage, we use the backend_config dictionary, which includes the client_resources key. This key specifies the amount of CPU and GPU resources each client can access.

In [ ]:
# Specify the resources each of your clients need
# By default, each client will be allocated 1x CPU and 0x GPUs
backend_config = {"client_resources": {"num_cpus": 1, "num_gpus": 0.0}}

# When running on GPU, assign an entire GPU for each client
if DEVICE.type == "cuda":
    backend_config = {"client_resources": {"num_cpus": 1, "num_gpus": 1.0}}
    # Refer to our Flower framework documentation for more details about Flower simulations
    # and how to set up the `backend_config`

### Launching the Simulation

With both ClientApp and ServerApp ready, we can launch the simulation. The `run_simulation` function is used to execute the simulation. This function orchestrates the federated learning simulation by bringing together the `ServerApp`, `ClientApp`, and other configuration details.

In [ ]:
from flwr.simulation import run_simulation

run_simulation(
    server_app=server_app,
    client_app=client_app,
    num_supernodes=NUM_PARTITIONS,
    backend_config=backend_config,
)

### Model Evaluation

The classification report provides detailed metrics for each class, including precision, recall, and F1-score.

In [ ]:
print("Classification Report", end="\n\n")
print(classification_report(test_output_true, test_output_pred, target_names=labels))

A confusion matrix helps us understand the types of misclassifications made by the model.

In [ ]:
plot_confusion_matrix(y_true=test_output_true,
                      y_pred=test_output_pred,
                      labels=labels)

This analysis helps identify areas where the model may need improvement, such as handling class imbalances or misclassifications. 